In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

from lib.data.datareader_total import DataReaderTotal
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.utils.args import get_opts_args, get_opt_args_from_model_name
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.data.dataset_motion_3d import MotionDataset3DTotal
from lib.model.evaluation import *

In [2]:
def get_notion_dicts(NOTION_API_KEY, DATABASE_ID):
    pages = get_all_database_pages(NOTION_API_KEY, DATABASE_ID)
    
    pages_dict = {}
    for page in pages:
        pages_dict[page["id"]] = page
        
    model_dict = {}
    for page in pages:
        page_id = page["id"]
        property = page["properties"]
        try:
            model_name = property["Model"]['select']['name']
        except: continue
        if model_name not in model_dict:
            model_dict[model_name] = []
        model_dict[model_name].append(page_id)
    return pages_dict, model_dict

def get_all_database_pages(NOTION_API_KEY, DATABASE_ID):    
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
    all_pages = []
    has_more = True
    start_cursor = None
    
    while has_more:
        if start_cursor:
            data = {"start_cursor": start_cursor}
        else:
            data = {}
        
        response = requests.post(url, headers=headers, json=data)
        response_data = response.json()
        
        all_pages.extend(response_data["results"])
        has_more = response_data.get("has_more", False)
        start_cursor = response_data.get("next_cursor", None)
    print(f"Total pages: {len(all_pages)}")
    return all_pages

def update_mpjpe(page_id, mpjpe, p_mpjpe):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    data = {
        "properties": {
            "MPJPE": {
                "number": mpjpe
            },
            "P-MPJPE": {
                "number": p_mpjpe
            }
        },
    }

    response = requests.patch(url, headers=headers, json=data)
    return response.json()

def create_notion_page(database_id, properties, children=None):
    url = "https://api.notion.com/v1/pages"
    data = {
        "parent": {"database_id": database_id},
        "properties": properties
    }
    if children:
        data["children"] = children
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()

def update_notion_page(page_id, properties, children=None):
    print(f"Updating page {page_id}")
    url = f"https://api.notion.com/v1/pages/{page_id}"
    data = {
        "properties": properties
    }
    if children:
        data["children"] = children
    
    response = requests.patch(url, headers=headers, json=data)
    return response.json()

In [3]:
import requests
from tqdm import tqdm
import copy

# Notion API 키 및 데이터베이스 ID 설정
notion = readpkl('notion.pkl')
NOTION_API_KEY = notion['NOTION_API_KEY']
DATABASE_ID = notion['DATABASE_ID']

# 헤더 설정
headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

pages_dict, model_dict = get_notion_dicts(NOTION_API_KEY, DATABASE_ID)

Total pages: 262


### Check experiment page existence

In [4]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_tr_s1_ts_s5678',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist_input_centering_tr_s1_ts_s5678',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist_s15678_tr_54138969_ts_others',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist_tr_s1_ts_s5678',
                        'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5',
                        ]
experiment_list = [item.split('.')[0] for item in os.listdir('experiments') if item not in blacklist_checkpoint]


In [5]:
# check to-make list
to_make_parent = []
to_make_child = {}
for experiment in experiment_list:
    child_list = []
    with open(f'experiments/{experiment}.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            child_list.append(line.split('\n')[0].strip())
    num_childs = len(child_list)
    if experiment not in model_dict.keys():
        #print(f'\n{experiment} is not in the model_dict')
        to_make_parent.append(experiment)
        to_make_child[experiment] = child_list
    else:
        model_ids = model_dict[experiment]
        if len(model_ids) > 1: # if there is a child
            # check parent id
            for model_id in model_ids:
                is_parent = pages_dict[model_id]['properties']['Is_parent']['checkbox']
                if is_parent: 
                    parent_id = model_id
                    break
            child_ids = [model_id for model_id in model_ids if model_id != parent_id]
            child_page_list = [pages_dict[child_id]['properties']['Name']['title'][0]['text']['content'] for child_id in child_ids]
            if len(child_page_list) < num_childs: to_make_child[experiment] = list(set(child_list) - set(child_page_list))
        else:
            parent_id = model_ids[0]
            child_ids = []
            child_page_list = []
            to_make_child[experiment] = child_list

In [6]:
to_make_parent, to_make_child

(['MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678_frame_81',
  'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678_frame_81'],
 {'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678_frame_81': ['3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN',
   '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6',
   'FIT3D-GT-CAM_NO_FACTOR-ALL_TEST',
   'H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678'],
  'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678_frame_81': ['3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN',
   '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6',
   'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z',
   'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST',
   'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678']})

In [7]:
for checkpoint in to_make_parent:
    # create new page
    if 'MB' in checkpoint: architecture = 'MotionBERT'
    else: architecture = None
    if 'frame_27' in checkpoint: input_frame = 27
    elif 'frame_81' in checkpoint: input_frame = 81
    else: input_frame = 243
    
    train_dataset = ''
    test_dataset = ''
    if 'h36m' in checkpoint: 
        train_dataset += 'H36M '
        test_dataset += 'H36M '
        
        if 'tr_s1' in checkpoint: 
            train_dataset += 'S1'
            if 'ts_s5678' in checkpoint: test_dataset += 'S5 6 7 8'
            else: test_dataset += 'ALL EXCEPT S1'   
        elif 's15678_tr_54138969' in checkpoint: 
            train_dataset += 'S1 5 6 7 8 CAM 54138969'
            test_dataset += 'S1 5 6 7 8 CAM EXCEPT 54138969'
        else: 
            train_dataset += 'S1 5 6 7 8'
            test_dataset += 'S9 11'
    elif 'fit3d' in checkpoint: 
        train_dataset += 'FIT3D '
        test_dataset += 'FIT3D '
        if 'tr_s03' in checkpoint: 
            train_dataset += 'S3'
            test_dataset += 'ALL TRAIN EXCEPT S3'
        elif 'ts_s4710' in checkpoint: 
            train_dataset += 'S3 5 8 9 11'
            test_dataset += 'S4 7 10'
    elif '3dhp' in checkpoint: 
        train_dataset = None
        test_dataset = None
    
    parent_properties = {
        "Name": {"title": [{'type': 'text', 'text': {'content': checkpoint}}]},
        #"Model": {"select": {"name": checkpoint}},
        "Is_parent": {"checkbox": True},
        "Architecture": {"select": {"name": architecture}},
        "Input frame": {"number": input_frame},
        "Train dataset": {"select": {"name": train_dataset}},
    }
    res = create_notion_page(DATABASE_ID, parent_properties, children=None)
    print(res)
    
# Reload notion pages
pages_dict, model_dict = get_notion_dicts(NOTION_API_KEY, DATABASE_ID)

{'object': 'page', 'id': '8fb4d042-4b57-4420-839a-b740fdc5e651', 'created_time': '2024-08-05T10:27:00.000Z', 'last_edited_time': '2024-08-05T10:27:00.000Z', 'created_by': {'object': 'user', 'id': 'c6d38448-0013-4c0a-91c8-359f6979bf12'}, 'last_edited_by': {'object': 'user', 'id': 'c6d38448-0013-4c0a-91c8-359f6979bf12'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '697fd4ef-9dca-42f7-a204-bd1b57e64b83'}, 'archived': False, 'in_trash': False, 'properties': {'Input frame': {'id': '%3BvPG', 'type': 'number', 'number': 81}, 'Train 2D input': {'id': '%3FY%7BB', 'type': 'formula', 'formula': {'type': 'string', 'string': 'SH'}}, 'P-MPJPE_prev': {'id': 'Avig', 'type': 'number', 'number': None}, 'Parent item': {'id': 'FQBZ', 'type': 'relation', 'relation': [], 'has_more': False}, 'Process': {'id': 'FiF%5E', 'type': 'select', 'select': None}, 'MPJPE_prev': {'id': 'KC%3C%7D', 'type': 'number', 'number': None}, 'Input scaling': {'id': 'K%5EVC', 'type': 'select', 'se

In [11]:
for checkpoint in to_make_child.keys():
    model_ids = model_dict[checkpoint]
    for model_id in model_ids:
        is_parent = pages_dict[model_id]['properties']['Is_parent']['checkbox']
        if is_parent: 
            parent_id = model_id
            break
    parent_properties = pages_dict[parent_id]['properties']
    model = parent_properties['Model']
    architecture = parent_properties['Architecture']
    input_frame = parent_properties['Input frame']
    input_scaling = parent_properties['Input scaling']
    canonical = parent_properties['Canonical']
    train_dataset = parent_properties['Train dataset']
    
    child_list = to_make_child[checkpoint]
    for subset in child_list:
        child_properties = {
            "Name": {"title": [{"text": {"content": subset}}]},
            #"Model": model,
            "Is_parent": {"checkbox": False},
            "Architecture": architecture,
            "Canonical": canonical,
            "Input frame": input_frame,
            "Input scaling": input_scaling,
            "Train dataset": train_dataset,
            "Parent item": {'id': 'FQBZ', 'type': 'relation', 'relation': [{'id': parent_id}], 'has_more': False}
        }
        res = create_notion_page(DATABASE_ID, child_properties, children=None)
        print(res)

{'object': 'page', 'id': 'd18bf926-1556-4af4-8467-16f5415a229d', 'created_time': '2024-08-04T09:45:00.000Z', 'last_edited_time': '2024-08-04T09:45:00.000Z', 'created_by': {'object': 'user', 'id': 'c6d38448-0013-4c0a-91c8-359f6979bf12'}, 'last_edited_by': {'object': 'user', 'id': 'c6d38448-0013-4c0a-91c8-359f6979bf12'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '697fd4ef-9dca-42f7-a204-bd1b57e64b83'}, 'archived': False, 'in_trash': False, 'properties': {'Input frame': {'id': '%3BvPG', 'type': 'number', 'number': 243}, 'Train 2D input': {'id': '%3FY%7BB', 'type': 'formula', 'formula': {'type': 'string', 'string': 'SH'}}, 'P-MPJPE_prev': {'id': 'Avig', 'type': 'number', 'number': None}, 'Parent item': {'id': 'FQBZ', 'type': 'relation', 'relation': [{'id': '65cc6389-36bb-4b8d-8435-21a576fad43b'}], 'has_more': False}, 'Process': {'id': 'FiF%5E', 'type': 'select', 'select': None}, 'MPJPE_prev': {'id': 'KC%3C%7D', 'type': 'number', 'number': None}, 'Input s

In [8]:
result_dict = readpkl('result_dict.pkl')

### Compare before and after 

In [4]:
subset_dict = {}
for checkpoint in result_dict.keys():
    if 'cam_no_factor' not in checkpoint: continue
    print(f'\n{checkpoint}')
    model_ids = model_dict[checkpoint]
    for model_id in model_ids:
        page = pages_dict[model_id]
        train = page['properties']['Train dataset']['select'] 
        test = page['properties']['Test dataset']['select']
        # filtering - parent
        if test == None: continue # parent
        
        # get child page's info
        child_page = pages_dict[model_id]
        child_properties = child_page['properties']
        if len(child_properties['Name']['title']) == 0: continue
        child_title = child_properties['Name']['title'][0]['text']['content']
        if child_title in result_dict[checkpoint].keys():
            mpjpe = child_properties['MPJPE_prev']['number']
            p_mpjpe = child_properties['P-MPJPE_prev']['number']
            if mpjpe == None: mpjpe = -1
            if p_mpjpe == None: p_mpjpe = -1
            if len(result_dict[checkpoint][child_title].keys()) >0 :
                #print(f'\t{child_title}')
                e1 = result_dict[checkpoint][child_title]["e1"]
                e2 = result_dict[checkpoint][child_title]["e2"]
                if e1 == None: e1 = -1
                if e2 == None: e2 = -1
            else:
                e1 = -1
                e2 = -1
            if e1 > 0 and mpjpe > 0:
                if e1 - mpjpe > 1: mark = '⬆️'
                elif e1 - mpjpe < -1: mark = '⬇️'
                else: mark = '✅'
            else:
                mark = '❌'
            print(f'{mark}\t{e1:.2f} {e2:.2f} <- {mpjpe:.2f} {p_mpjpe:.2f}\t{child_title}')
            if child_title not in subset_dict: subset_dict[child_title] = []
            subset_dict[child_title].append(mark)


MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710
✅	159.42 117.13 <- 159.63 117.19	3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
⬇️	167.25 121.53 <- 288.42 154.17	3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
✅	113.02 84.69 <- 113.02 84.69	H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
✅	108.62 79.20 <- 108.62 79.20	H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678
✅	30.49 14.50 <- 30.49 14.50	FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710

MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03
❌	186.13 125.26 <- -1.00 -1.00	3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN
❌	177.16 118.93 <- -1.00 -1.00	3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6
❌	41.92 26.26 <- -1.00 -1.00	FIT3D-GT-CAM_NO_FACTOR-TR_S03
❌	130.98 94.77 <- -1.00 -1.00	H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678
❌	133.72 100.37 <- -1.00 -1.00	H36M-GT-CAM_NO_FACTOR

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s03

KeyError: 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering_tr_s1_ts_s5678_frame_27'

In [9]:
for subset in natsorted(subset_dict):
    print(subset, subset_dict[subset])

3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_ALL_TRAIN ['❌', '❌']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_TS1_6 ['❌', '❌']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN ['✅', '⬆️', '⬇️', '✅', '⬆️', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6 ['⬇️', '⬆️', '⬆️', '⬆️', '⬇️', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN ['⬇️', '⬇️', '⬇️', '⬇️', '✅', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6 ['⬇️', '⬇️', '⬆️', '⬇️', '⬇️', '⬇️', '⬇️', '⬆️']
FIT3D-GT-CAM_NO_FACTOR-ALL_TEST ['⬇️', '⬇️', '⬇️', '⬇️', '⬇️']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-ALL_TEST ['✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST ['✅', '⬇️', '✅', '✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S03 ['⬇️', '⬇️']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710 ['✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-TR_S03 ['⬇️']
FIT3D-GT-CAM_NO_FACTO

### Update notion pages

In [9]:
for checkpoint in result_dict.keys():
    if 'cam_no_factor' not in checkpoint: continue
    print(f'\n{checkpoint}')
    model_ids = model_dict[checkpoint]
    if len(model_ids) > 1: # if there is a child
        # check parent id
        for model_id in model_ids:
            page = pages_dict[model_id]
            is_parent = page['properties']['Is_parent']['checkbox']
            if is_parent: 
                parent_id = model_id
                break
        
        for model_id in model_ids:  
            if model_id == parent_id: continue # filtering - parent
            # get child page's info
            child_page = pages_dict[model_id]
            child_properties = child_page['properties']
            if len(child_properties['Name']['title']) == 0: continue
            child_title = child_properties['Name']['title'][0]['text']['content']
            child_mpjpe = child_properties['MPJPE']['number']
            child_p_mpjpe = child_properties['P-MPJPE']['number']
            if child_mpjpe != None and child_p_mpjpe != None: continue
            
            if child_title in result_dict[checkpoint].keys():
                e1 = result_dict[checkpoint][child_title]["e1"]
                e2 = result_dict[checkpoint][child_title]["e2"]
                if e1 == None: e1 = -1
                if e2 == None: e2 = -1
                update_mpjpe(model_id, float(f'{e1:.2f}'), float(f'{e2:.2f}'))
            


MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710

MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s03

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710

MB_train_h36m_gt_cam_no_factor

MB_train_fit3d_gt_cam_no_factor_ts_s4710

MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering

MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_tr_s03

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering_tr_s1_ts_s5678

MB_train_h36m_gt_cam_no_factor_s15678_tr_54138969_ts_others

MB_train_fit3d_gt_cam_no_factor_tr_s03

MB_train_h36m_gt_cam_no_factor_with_canonical2

MB_train_h3

### Check test dataset

In [10]:
# Reload notion pages
pages_dict, model_dict = get_notion_dicts(NOTION_API_KEY, DATABASE_ID)

Total pages: 262


In [14]:
for experiment in experiment_list:
    model_ids = model_dict[experiment]
    for model_id in model_ids:
        page = pages_dict[model_id]
        try:
            test = page['properties']['Test dataset']['select']['name']
        except:
            print(f'{experiment} has no test dataset')

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678 has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678 has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678 has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678 has no test dataset
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710 has no test dataset
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710 has no test dataset
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_tr_s03 has no test dataset
MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03 has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_